# <span style="color:orange">11/05 - Pair Programming Limpieza 6 - Lidia y Tatiana</span>.

# Ejercicio 1
Lo primero que debemos hacer es identificar cuáles de nuestras columnas es categóricas. Extraed el nombre de las columnas que contienen variables categóricas.

In [37]:
# antes de empezar importemos las librerías que necesitaremos

import pandas as pd
from sklearn.preprocessing import LabelEncoder # para realizar el Label Encoding 
from sklearn.preprocessing import OneHotEncoder  # para realizar el One-Hot Encoding

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

import numpy as np


In [38]:
df = pd.read_csv("./attacks_limpiando_con_coordenadas.csv", index_col = 0)

In [39]:
#vamos a obtener nuestras variables categoricas
categoricas = df.select_dtypes(include = "object")
categoricas.head(2)

,type,country,species_,fecha_limpia,fatal,sex,latitud,longitud
0,Boating,usa,White shark,Jun,N,F,39.7837304,-100.445882
1,Unprovoked,usa,Unespecific,Jun,N,F,39.7837304,-100.445882


# Ejercicio 2 
Si nos fijamos en la columna de country es my complicado codificar esa columna ya que hay muchísimas categorías. Para ello, vamos a usar una librería que llamada geopy que nos devuelve las coordenadas de un pais, ciudad, pueblo, lo que imaginemos! El objetivo de esto es poder clasificar nuestros paises en Hemisferio Norte y Sur para poder hacer una codificación más eficaz. Os dejamos por aquí la documentación de la librería para que veais como funciona y el código:

-------------

In [40]:
pip install geopy # instalamos la libreria que nos permite referenciar los paises

Note: you may need to restart the kernel to use updated packages.


In [41]:
def extraer_coordenadas(x):
    try:
        geolocator = Nominatim(user_agent = "ana")
        location = geolocator.geocode(x)
        return location.lat, location.long
    except:
        return "no hay datos"

----------------

Como ya tenemos las coordenadas, deberéis codificar la columna en función de si pertenecen al Hemisferio Norte (HN) o Sur (HS). Aquellas que pertencezcan al HN le pondremos un 1 y a las del HS le pondremos un 0.

📌 Nota Pertenezeran al HN las que su latitud sea positiva y las del HS las que su latitud sea negativa.

In [42]:
df_copia = df.copy()

In [43]:
df.head(3)


,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.7837304,-100.445882
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.7837304,-100.445882
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.7837304,-100.445882


In [44]:
df_copia["latitud"].dtypes

dtype('O')

In [45]:
df_copia.latitud.unique()

array(['39.7837304', '-24.7761086', '23.6585116', '-10.3333333',
       '52.5310214', '-28.8166236', '14.8971921', '10.2735633',
       '4.7064352', '24.7736546', '-21.3019905', '-1.3397668',
       '4.5693754', '26.8234472', '25.029422', '23.0131338',
       '-20.2759451', '-41.5000831', '39.3260685', '-13.7693895',
       '-8.7053941', '36.5748441', '26.2540493', 'n', '-12.2045176',
       '-21.130737949999997', '-16.03442485', '54.7023545', '24.0002488',
       '12.7503486', '-2.4833826', '35.000074', '4.099917', '16.0000552',
       '-18.1239696', '19.0974031', '19.703182249999998', '12.51756625',
       '-19.302233', '18.2214149', '42.6384261', '13.581921',
       '38.9953683', '48.3500473', '46.603354', '-5.6816069',
       '10.8677845', '0.3448612', '31.5313113', '-7.338358449999999',
       '23.9739374', '18.1850507', '31.462420950000002',
       '13.450125700000001', '-4.6574977', '16.8259793', '9.6000359',
       '-19.9160819', '56.7861112', '61.0666922', '45.5643442',
      

In [46]:
#Creamos una función para determinar según latitud los paises del Hemisferio Sur o Norte
def seleccion_hemisferio(col):
    for x in col:
        if x >= "0":
            return "HN"
        elif x < "0":
            return "HS"
        else:
            return "unknown"


In [47]:
#Y aplicamos la función, creando una nueva columna llamada hemisferio.
df_copia["hemisferio"] = df_copia["latitud"].apply(seleccion_hemisferio)

In [48]:
# Aqui comprbamos que seha ejecutado correctamente
df_copia.hemisferio.unique()

array(['HN', 'HS'], dtype=object)

In [49]:
df_copia.head()

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,hemisferio
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.7837304,-100.445882,HN
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.7837304,-100.445882,HN
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.7837304,-100.445882,HN
3,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.7761086,134.755,HS
4,2018,Provoked,mexico,27.0,Tiger shark,Jun,N,M,23.6585116,-102.0077097,HN


    Aplicamos one_hot_encoder,creará una columna para cada valor distinto que exista en la característica que estamos codificando, para cada registro, marcar con un 1 la columna a la que pertenezca dicho registro y dejar las demás con 0, en este caso una para hemisferio sur y otra para hemisferio Norte.

In [50]:
# vamos a definir una función que nos aplique este método

def one_hot_encoder_one(df_copia,columna,keep_first=True):
    
    # iniciamos el método de OneHot Encoder
    oh = OneHotEncoder()
    
    # hacemos la codificación de los datos 
    transformados = oh.fit_transform(df_copia[[columna]])
    
    # convertimos nuestro array con la codificación hecha en un dataframe
    oh_df = pd.DataFrame(transformados.toarray())
    
    # el método get_feature_names nos va a dar el nombre de las columnas nuevas que se nos generarán
    oh_df.columns = oh.get_feature_names()
    
    # concatenamos el dataframe original con el dataframe que acabamos de crear
    final = pd.concat([df_copia,oh_df],axis=1)
    
    # eliminamos la columna original 
    final.drop(columna, axis = 1,  inplace = True)
    return final

In [51]:
df_copia.head()

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,hemisferio
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.7837304,-100.445882,HN
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.7837304,-100.445882,HN
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.7837304,-100.445882,HN
3,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.7761086,134.755,HS
4,2018,Provoked,mexico,27.0,Tiger shark,Jun,N,M,23.6585116,-102.0077097,HN


In [52]:
# aplicamos nuestra función al dataset para que nos haga el encoding sobre la columna "plataforms
df_copia = one_hot_encoder_one(df_copia, "hemisferio")

In [53]:
df_copia.head()

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,x0_HN,x0_HS
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.7837304,-100.445882,1.0,0.0
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.7837304,-100.445882,1.0,0.0
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.7837304,-100.445882,1.0,0.0
3,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.7761086,134.755,0.0,1.0
4,2018,Provoked,mexico,27.0,Tiger shark,Jun,N,M,23.6585116,-102.0077097,1.0,0.0


In [54]:
df_copia.rename(columns={"x0_HN": "Hemisferio Norte", "x0_HS": "Hemisferio Sur"},inplace=True)
df_copia.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,Hemisferio Norte,Hemisferio Sur
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.7837304,-100.445882,1.0,0.0
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.7837304,-100.445882,1.0,0.0


# Ejercicio 3
Aplicad un get_dummies a las columnas de fataly age.


    Get_dummies:Nos permite eliminar la primera de las columnas generadas para cada característica codificada para evitar la denominada colinealidad (que una de las características sea una combinación lineal de las otras), lo que dificulta el correcto funcionamiento de los algoritmos.

In [55]:
df_copia.head(3)


,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,Hemisferio Norte,Hemisferio Sur
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.7837304,-100.445882,1.0,0.0
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.7837304,-100.445882,1.0,0.0
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.7837304,-100.445882,1.0,0.0


In [56]:
df.fatal.unique()

array(['N', 'Y', 'Unknown'], dtype=object)

    Podemos crear una función para get_dummies, aqui como prefijo en las nuevas columnas que se crearán según los únicos de la columna, tendrán el mismo nombre que la columna.

In [57]:
def get_dummies(df_copia,columna,keep_first=True):

    dummies = pd.get_dummies(df_copia[columna], prefix_sep = "_", prefix = [columna], dtype = int)
    df_dummies = pd.concat([df_copia, dummies], axis = 1)
    return df_dummies

In [58]:
df_dummies_fatal = get_dummies(df_copia, "fatal")

In [59]:
df_dummies_fatal.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,Hemisferio Norte,Hemisferio Sur,['fatal']_N,['fatal']_Unknown,['fatal']_Y
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.7837304,-100.445882,1.0,0.0,1,0,0
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.7837304,-100.445882,1.0,0.0,1,0,0


In [60]:
df_dummies_age = get_dummies(df_dummies_fatal, "age")

In [61]:
df_dummies_age.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,Hemisferio Norte,Hemisferio Sur,['fatal']_N,['fatal']_Unknown,['fatal']_Y,['age']_1.0,['age']_2.0,['age']_3.0,['age']_5.0,['age']_6.0,['age']_7.0,['age']_8.0,['age']_9.0,['age']_10.0,['age']_11.0,['age']_12.0,['age']_13.0,['age']_14.0,['age']_15.0,['age']_16.0,['age']_17.0,['age']_18.0,['age']_19.0,['age']_20.0,['age']_21.0,['age']_22.0,['age']_23.0,['age']_24.0,['age']_25.0,['age']_26.0,['age']_27.0,['age']_28.0,['age']_29.0,['age']_30.0,['age']_31.0,['age']_32.0,['age']_33.0,['age']_34.0,['age']_35.0,['age']_36.0,['age']_37.0,['age']_38.0,['age']_39.0,['age']_40.0,['age']_41.0,['age']_42.0,['age']_43.0,['age']_44.0,['age']_45.0,['age']_46.0,['age']_47.0,['age']_48.0,['age']_49.0,['age']_50.0,['age']_51.0,['age']_52.0,['age']_53.0,['age']_54.0,['age']_55.0,['age']_56.0,['age']_57.0,['age']_58.0,['age']_59.0,['age']_60.0,['age']_61.0,['age']_62.0,['age']_63.0,['age']_64.0,['age']_65.0,['age']_66.0,['age']_67.0,['age']_68.0,['age']_69.0,['age']_70.0,['age']_71.0,['age']_72.0,['age']_73.0,['age']_74.0,['age']_75.0,['age']_77.0,['age']_78.0,['age']_81.0,['age']_82.0,['age']_84.0,['age']_86.0,['age']_87.0
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.7837304,-100.445882,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.7837304,-100.445882,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


----

    También podemos verlo como los ejercicios de clase, de esta forma individual por ejemplo, podemos definir el prefijo para cada columna, si queremos que sea diferente para cada uno.

In [62]:
dummies = pd.get_dummies(df_copia["fatal"], prefix_sep = "_", prefix = "género", dtype = int)
dummies.head(2)

,género_N,género_Unknown,género_Y
0,1,0,0
1,1,0,0


In [63]:
df_dummies = pd.concat([df_copia, dummies], axis = 1)
df_dummies.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,Hemisferio Norte,Hemisferio Sur,género_N,género_Unknown,género_Y
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.7837304,-100.445882,1.0,0.0,1,0,0
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.7837304,-100.445882,1.0,0.0,1,0,0


In [64]:
dummies_age = pd.get_dummies(df_copia["age"], prefix_sep = "_", prefix = "edad", dtype = int)
dummies_age.head(2)

,edad_1.0,edad_2.0,edad_3.0,edad_5.0,edad_6.0,edad_7.0,edad_8.0,edad_9.0,edad_10.0,edad_11.0,edad_12.0,edad_13.0,edad_14.0,edad_15.0,edad_16.0,edad_17.0,edad_18.0,edad_19.0,edad_20.0,edad_21.0,edad_22.0,edad_23.0,edad_24.0,edad_25.0,edad_26.0,edad_27.0,edad_28.0,edad_29.0,edad_30.0,edad_31.0,edad_32.0,edad_33.0,edad_34.0,edad_35.0,edad_36.0,edad_37.0,edad_38.0,edad_39.0,edad_40.0,edad_41.0,edad_42.0,edad_43.0,edad_44.0,edad_45.0,edad_46.0,edad_47.0,edad_48.0,edad_49.0,edad_50.0,edad_51.0,edad_52.0,edad_53.0,edad_54.0,edad_55.0,edad_56.0,edad_57.0,edad_58.0,edad_59.0,edad_60.0,edad_61.0,edad_62.0,edad_63.0,edad_64.0,edad_65.0,edad_66.0,edad_67.0,edad_68.0,edad_69.0,edad_70.0,edad_71.0,edad_72.0,edad_73.0,edad_74.0,edad_75.0,edad_77.0,edad_78.0,edad_81.0,edad_82.0,edad_84.0,edad_86.0,edad_87.0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [65]:
df_dummies = pd.concat([df_dummies, dummies_age], axis = 1)
df_dummies.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,Hemisferio Norte,Hemisferio Sur,género_N,género_Unknown,género_Y,edad_1.0,edad_2.0,edad_3.0,edad_5.0,edad_6.0,edad_7.0,edad_8.0,edad_9.0,edad_10.0,edad_11.0,edad_12.0,edad_13.0,edad_14.0,edad_15.0,edad_16.0,edad_17.0,edad_18.0,edad_19.0,edad_20.0,edad_21.0,edad_22.0,edad_23.0,edad_24.0,edad_25.0,edad_26.0,edad_27.0,edad_28.0,edad_29.0,edad_30.0,edad_31.0,edad_32.0,edad_33.0,edad_34.0,edad_35.0,edad_36.0,edad_37.0,edad_38.0,edad_39.0,edad_40.0,edad_41.0,edad_42.0,edad_43.0,edad_44.0,edad_45.0,edad_46.0,edad_47.0,edad_48.0,edad_49.0,edad_50.0,edad_51.0,edad_52.0,edad_53.0,edad_54.0,edad_55.0,edad_56.0,edad_57.0,edad_58.0,edad_59.0,edad_60.0,edad_61.0,edad_62.0,edad_63.0,edad_64.0,edad_65.0,edad_66.0,edad_67.0,edad_68.0,edad_69.0,edad_70.0,edad_71.0,edad_72.0,edad_73.0,edad_74.0,edad_75.0,edad_77.0,edad_78.0,edad_81.0,edad_82.0,edad_84.0,edad_86.0,edad_87.0
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.7837304,-100.445882,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.7837304,-100.445882,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Ejercicio 4 
Aplicad un Label Encoding a la columna species , fecha y type.

In [66]:
df_label = df_dummies.copy()
df_label.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,Hemisferio Norte,Hemisferio Sur,género_N,género_Unknown,género_Y,edad_1.0,edad_2.0,edad_3.0,edad_5.0,edad_6.0,edad_7.0,edad_8.0,edad_9.0,edad_10.0,edad_11.0,edad_12.0,edad_13.0,edad_14.0,edad_15.0,edad_16.0,edad_17.0,edad_18.0,edad_19.0,edad_20.0,edad_21.0,edad_22.0,edad_23.0,edad_24.0,edad_25.0,edad_26.0,edad_27.0,edad_28.0,edad_29.0,edad_30.0,edad_31.0,edad_32.0,edad_33.0,edad_34.0,edad_35.0,edad_36.0,edad_37.0,edad_38.0,edad_39.0,edad_40.0,edad_41.0,edad_42.0,edad_43.0,edad_44.0,edad_45.0,edad_46.0,edad_47.0,edad_48.0,edad_49.0,edad_50.0,edad_51.0,edad_52.0,edad_53.0,edad_54.0,edad_55.0,edad_56.0,edad_57.0,edad_58.0,edad_59.0,edad_60.0,edad_61.0,edad_62.0,edad_63.0,edad_64.0,edad_65.0,edad_66.0,edad_67.0,edad_68.0,edad_69.0,edad_70.0,edad_71.0,edad_72.0,edad_73.0,edad_74.0,edad_75.0,edad_77.0,edad_78.0,edad_81.0,edad_82.0,edad_84.0,edad_86.0,edad_87.0
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.7837304,-100.445882,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.7837304,-100.445882,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [77]:
#iniciamos el método
le = LabelEncoder()

#lo aplicamos a todo nuestro dataframe 
for col in df_label.select_dtypes(include='object').columns:
    df_label[col]=le.fit_transform(df_label[col])

# Ejercicio 5 
Guardad los cambios en un csv nuevo para seguir trabajando con el en siguientes ejercicios de pair.

In [78]:
df_label.to_csv("Limpieza_6_encoding.csv")